# Mangadex

The command below is used to run on Window, Linux will have different command

In [ ]:
import os
import subprocess


def download_test_set(path_to_dataset_folder="./"):
    vi_folder = os.path.join(path_to_dataset_folder, "vi")
    en_folder = os.path.join(path_to_dataset_folder, "en")
    os.makedirs(vi_folder, exist_ok=True)
    os.makedirs(en_folder, exist_ok=True)
    print("Downloading images from MangaDex...")

    for order, link in enumerate(mangadex_links):
        for lang, out_dir in [("en", en_folder), ("vi", vi_folder)]:
            args = [
                "mangadex-dl",
                link,
                "--path",
                os.path.join(out_dir, "{manga.title}"),
                "--no-group-name",
                "--replace",
                "--log-level",
                "WARNING",
                "--use-compressed-image",
                "--cover",
                "none",
                "--language",
                lang,
                "--delay-requests",
                "0.75",
            ]
            try:
                subprocess.run(args, check=True)
                print(
                    f"Finsihed {lang.upper()} {order+1}/{len(mangadex_links)}: {link}"
                )
                continue
            except subprocess.CalledProcessError as e:
                print(
                    f"Error {lang.upper()} {order+1}/{len(mangadex_links)}: {link} {e}"
                )
                continue

    print(f"Manga saved to {path_to_dataset_folder}")


mangadex_links = [
    "https://mangadex.org/title/0a2fd70d-e4de-49fb-8e81-89311d46b329/ruri-dragon-oneshot",
    "https://mangadex.org/title/63f2efed-e0cf-4ad1-98a9-7fba7f5848c1/the-raise",
]

saved_folder = "../data"
download_test_set(path_to_dataset_folder=saved_folder)

Finsihed EN 1/1: https://mangadex.org/title/0a2fd70d-e4de-49fb-8e81-89311d46b329/ruri-dragon-oneshot
Finsihed VI 1/1: https://mangadex.org/title/0a2fd70d-e4de-49fb-8e81-89311d46b329/ruri-dragon-oneshot
Manga saved to ../data


CLi version

In [ ]:
!mangadex-dl https://mangadex.org/title/63f2efed-e0cf-4ad1-98a9-7fba7f5848c1/the-raise --language "vi" --path "./VN/{manga.title}" --cover "none"

[INFO] Checking url = https://mangadex.org/title/63f2efed-e0cf-4ad1-98a9-7fba7f5848c1/the-raise
[INFO] Using Vietnamese language
[INFO] Fetching all chapters...
[INFO] Download directory is set to "D:\Downloads\Tu_Lieu\Cao_Hoc\Master_Thesis\master-thesis\utils\VN\The Raise"
[INFO] Using raw format
[INFO] Downloading cover manga The Raise
[INFO] Not downloading cover manga, since "cover" is none
[INFO] Getting images from chapter 1
[INFO] Downloading [Senukin] Chapter. 1 page 1

File sizes:   0%|                                   | 0.00/1.55M [00:00<?, ?B/s]
File sizes:  13%|███▍                       | 197k/1.55M [00:00<00:01, 1.19MB/s]
File sizes:  36%|█████████▊                 | 565k/1.55M [00:00<00:00, 2.34MB/s]
File sizes:  77%|███████████████████▉      | 1.19M/1.55M [00:00<00:00, 3.88MB/s]
File sizes: 100%|██████████████████████████| 1.55M/1.55M [00:00<00:00, 3.73MB/s]
[INFO] Getting images from chapter 2
[INFO] Downloading [Senukin] Chapter. 2 page 1

File sizes:   0%|          